In [2]:
# Install required packages in Colab or Jupyter
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [10]:
# Load your CSV data
df = pd.read_csv('dataset.csv')
# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

In [11]:
# Sort and select relevant columns
df = df.sort_values('Timestamp').reset_index(drop=True)
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

# Define the schema for Pathway
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

In [12]:
# Replay the CSV as a simulated stream
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Parse timestamp and extract day
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [13]:
# Add dummy columns for Model 2 features (replace with actual data if available)
data_with_features = data_with_time.with_columns(
    QueueLength = 5,        # constant dummy value
    Traffic = 3,            # constant dummy value
    IsSpecialDay = 0,       # binary flag
    VehicleTypeWeight = 1   # constant dummy value
)

import datetime


In [14]:
# Aggregate daily max values
daily_agg = (
    data_with_features.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        queue_max=pw.reducers.max(pw.this.QueueLength),
        traffic_max=pw.reducers.max(pw.this.Traffic),
        special_day_max=pw.reducers.max(pw.this.IsSpecialDay),
        vehicle_weight_max=pw.reducers.max(pw.this.VehicleTypeWeight),
    )
)

In [15]:
# --- Model 1: Baseline Linear Model ---
alpha = 0.5
base_price = 10

model1 = daily_agg.with_columns(
    price=base_price + alpha * (pw.this.occ_max / pw.this.cap)
)

In [16]:
# --- Model 2: Demand-Based Price Function ---
beta = 0.3
gamma = 0.2
delta = 0.1
epsilon = 0.4
lambda_ = 0.7

In [17]:
# Estimate a max demand for normalization (adjust as needed)
max_demand = (
    alpha * 1 + beta * 10 - gamma * 0 + delta * 1 + epsilon * 5
)  # Example: occ_max/cap=1, queue=10, traffic=0, special=1, vehicle=5

model2 = daily_agg.with_columns(
    demand=alpha * (pw.this.occ_max / pw.this.cap) +
           beta * pw.this.queue_max -
           gamma * pw.this.traffic_max +
           delta * pw.this.special_day_max +
           epsilon * pw.this.vehicle_weight_max,
    normalized_demand=(
        alpha * (pw.this.occ_max / pw.this.cap) +
        beta * pw.this.queue_max -
        gamma * pw.this.traffic_max +
        delta * pw.this.special_day_max +
        epsilon * pw.this.vehicle_weight_max
    ) / max_demand,
    price=base_price * (
        1 + lambda_ * (
            (
                alpha * (pw.this.occ_max / pw.this.cap) +
                beta * pw.this.queue_max -
                gamma * pw.this.traffic_max +
                delta * pw.this.special_day_max +
                epsilon * pw.this.vehicle_weight_max
            ) / max_demand
        )
    )
)

In [20]:
model2 = model2.with_columns(
    price=pw.apply(
        lambda x: max(min(x, 2 * base_price), 0.5 * base_price),
        pw.this.price
    )
)

In [21]:
# --- Visualization Setup ---
pn.extension()

def price_plotter(source, price_col="price", title=""):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=title,
        x_axis_type="datetime",
    )
    fig.line("t", price_col, source=source, line_width=2, color="navy")
    fig.circle("t", price_col, source=source, size=6, color="red")
    return fig

In [22]:

# Visualizations for both models
viz1 = model1.plot(lambda source: price_plotter(source, "price", "Model 1: Baseline Linear Price"), sorting_col="t")
viz2 = model2.plot(lambda source: price_plotter(source, "price", "Model 2: Demand-Based Price"), sorting_col="t")

In [23]:
# Display both plots in a Panel layout
pn.Column("# Parking Price Models", "## Model 1: Baseline Linear", viz1, "## Model 2: Demand-Based", viz2).servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Markdown(str)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [24]:
# Start the Pathway pipeline (in background)
%%capture --no-display
pw.run()

Output()